In [1]:
!pip install git+https://github.com/SoshiTanabeDS/pipeline-cross-validation.git@refactor-into-script

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Cloning https://github.com/SoshiTanabeDS/pipeline-cross-validation.git (to revision refactor-into-script) to /tmp/pip-req-build-gnhf0bpy
  Running command git clone --filter=blob:none --quiet https://github.com/SoshiTanabeDS/pipeline-cross-validation.git /tmp/pip-req-build-gnhf0bpy
  Running command git checkout -b refactor-into-script --track origin/refactor-into-script
  Switched to a new branch 'refactor-into-script'
  Branch 'refactor-into-script' set up to track remote branch 'refactor-into-script' from 'origin'.
  Resolved https://github.com/SoshiTanabeDS/pipeline-cross-validation.git to commit 61219519308e4d89c00046f610f6133a9abbd007
  Preparing metadata (setup.py) ... done
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import os

bucket = "pipeline-cross-validation"
prefix = "data"
src_path = f"s3://{bucket}/{prefix}"

In [3]:
def create_iter_steps(iteration):
    
    # Parameters
    num_class = 8
    task_name = f"train_iter{iteration}"
    src_train_path = os.path.join(src_path, f"iter{iteration}")
    src_test_path = os.path.join(src_path, "test/features")
    dst_model_path = os.path.join(src_path, f"model_train/iter{iteration}")
    dst_test_path = os.path.join(src_path, f"output/iter{iteration}")
    hyperparameters = {
        "num_class":num_class,
        "max_depth":"5",
        "eta":"0.2",
        "min_child_weight":"1",
        "objective":"multi:softmax",
        "eval_metric":"mlogloss",
        "num_round":"500",
        "tree_method":"gpu_hist",
        "early_stopping_rounds":"10",
        # "csv_weights": "1"
    }

    from pipeline_cv.xgboost import create_steps
    steps = create_steps(
        task_name,
        src_train_path,
        src_test_path,
        dst_model_path,
        dst_test_path,
        training_instance_type,
        model_instance_type,
        hyperparameters
    )
    
    return steps

In [4]:
from sagemaker.workflow.parameters import ParameterString

training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.p3.2xlarge"
)
model_instance_type = ParameterString(
    name="ModelInstanceType",
    default_value="ml.m5.large"
)


In [5]:
iterations = 4
whole_steps = []
for iteration in range(iterations):
    steps = create_iter_steps(iteration)
    whole_steps.extend(steps)

In [7]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = f"220716Pipeline3"

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        training_instance_type,
        model_instance_type
        ],
    steps=whole_steps,
)

In [9]:
from sagemaker import get_execution_role
pipeline.upsert(role_arn=get_execution_role())
execution = pipeline.start()

In [11]:
execution.list_steps()

[{'StepName': 'train_iter1',
  'StartTime': datetime.datetime(2022, 7, 16, 7, 22, 33, 965000, tzinfo=tzlocal()),
  'StepStatus': 'Executing',
  'AttemptCount': 0,
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-2:173863741403:training-job/pipelines-ld3u7wmvyu9p-train-iter1-58aw7xs4zw'}}},
 {'StepName': 'train_iter2',
  'StartTime': datetime.datetime(2022, 7, 16, 7, 22, 33, 965000, tzinfo=tzlocal()),
  'StepStatus': 'Starting',
  'AttemptCount': 0,
  'Metadata': {}},
 {'StepName': 'train_iter3',
  'StartTime': datetime.datetime(2022, 7, 16, 7, 22, 33, 965000, tzinfo=tzlocal()),
  'StepStatus': 'Starting',
  'AttemptCount': 0,
  'Metadata': {}},
 {'StepName': 'train_iter0',
  'StartTime': datetime.datetime(2022, 7, 16, 7, 22, 33, 964000, tzinfo=tzlocal()),
  'StepStatus': 'Executing',
  'AttemptCount': 0,
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-2:173863741403:training-job/pipelines-ld3u7wmvyu9p-train-iter0-th9cs0vqwl'}}}]

In [ ]:
execution.describe()